In [1]:
import pandas as pd

#Load the game data to pandas data frame
df_season_map  = pd.read_csv('../data/game_dates_season.csv')
df_raw = pd.read_csv('../data/nfl_games_pfr_raw.csv')
df_teams = pd.read_csv('../data/teams.csv')

In [2]:
print(df_raw.head())
print(df_raw.columns)
print(len(df_raw))

   Week  Day        Date    Time            Winner/tie Unnamed: 5  \
0     1  Thu  2025-09-04  8:20PM   Philadelphia Eagles        NaN   
1     1  Fri  2025-09-05  8:00PM  Los Angeles Chargers        NaN   
2     1  Sun  2025-09-07  1:00PM  Jacksonville Jaguars        NaN   
3     1  Sun  2025-09-07  1:00PM   Pittsburgh Steelers          @   
4     1  Sun  2025-09-07  1:00PM    Cincinnati Bengals          @   

            Loser/tie Unnamed: 7  PtsW  PtsL   YdsW  TOW   YdsL  TOL  
0      Dallas Cowboys   boxscore  24.0  20.0  302.0  0.0  307.0  1.0  
1  Kansas City Chiefs   boxscore  27.0  21.0  394.0  0.0  347.0  0.0  
2   Carolina Panthers   boxscore  26.0  10.0  378.0  1.0  255.0  3.0  
3       New York Jets   boxscore  34.0  32.0  271.0  0.0  394.0  1.0  
4    Cleveland Browns   boxscore  17.0  16.0  141.0  0.0  327.0  2.0  
Index(['Week', 'Day', 'Date', 'Time', 'Winner/tie', 'Unnamed: 5', 'Loser/tie',
       'Unnamed: 7', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL'],
      dtype=

In [3]:
# I'm the worst documentation writer ever. Thank heavens for Copilot!
#Columns: Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,,PtsW,PtsL,YdsW,TOW,YdsL,TOL
"""
This script processes and prepares game data for analysis by merging and transforming various dataframes.
Steps:
1. Load raw game data and team abbreviation mappings.
2. Merge game data with season mapping based on the 'Date' column.
3. Merge game data with team abbreviations for both winners and losers.
4. Check for any missing season data and print a message if any are found.
5. Drop unnecessary columns and rename columns for clarity.
6. Create new columns to identify home and away teams, winners and losers based on yards and turnovers, and calculate various margins and totals.
7. Add columns to indicate if the home team won and if there is an away win bonus.
Columns:
- Week: The week of the game.
- Day: The day of the game.
- Date: The date of the game.
- Time: The time of the game.
- Winner/tie: The winning team or if the game was a tie.
- Unnamed: 5: Indicates if the loser is the home team.
- Loser/tie: The losing team or if the game was a tie.
- PtsW: Points scored by the winning team.
- PtsL: Points scored by the losing team.
- YdsW: Yards gained by the winning team.
- TOW: Turnovers by the winning team.
- YdsL: Yards gained by the losing team.
- TOL: Turnovers by the losing team.
New Columns:
- Home Team: The home team based on the 'LoserIsHome' column.
- Away Team: The away team based on the 'LoserIsHome' column.
- Winner Yds: The team with more yards.
- Loser Yds: The team with fewer yards.
- Margin Yds: The absolute difference in yards between the two teams.
- Winner TO: The team with more turnovers.
- Loser TO: The team with fewer turnovers.
- Margin TO: The absolute difference in turnovers between the two teams.
- Margin Pts: The absolute difference in points between the two teams.
- Total Pts: The total points scored in the game.
- Margin Pct: The difference between 'Margin Pts' and 'Total Pts'.
- home_team_winner: Boolean indicating if the home team won.
- away_win_bonus: Bonus point if the away team won.
Note:
- Error handling for unmapped abbreviations needs to be added.
"""
df_games = df_raw
df_teams_abbr = df_teams[['Team Name','Team Abbr']]

df_games = pd.merge(df_games, df_season_map, on='Date', how='left')
df_games = pd.merge(df_games, df_teams_abbr, left_on='Winner/tie', right_on='Team Name', how='left')
df_games = pd.merge(df_games, df_teams_abbr, left_on='Loser/tie', right_on='Team Name', how='left')

if len(df_games[df_games['season'].isna()])==0:
    df_games[['season','week']] = df_games[['season','week']].round().astype(int)
    
else:
    print("***Some game dates not included in mapping file")
    print(df_games[df_games['season'].isna()])

df_games = df_games.drop(columns=['Unnamed: 7','Unnamed: 0','Team Name_x','Team Name_y']).rename(columns={'Unnamed: 5':'LoserIsHome', 'Winner/tie':'Winner','Loser/tie':'Loser', 'Team Abbr_x':'Winner Abbr','Team Abbr_y':'Loser Abbr'})
df_games['Home Team'] = df_games.apply(lambda row: row['Loser Abbr'] if row['LoserIsHome']=="@" else row['Winner Abbr'], axis=1)
df_games['Away Team'] = df_games.apply(lambda row: row['Winner Abbr'] if row['LoserIsHome']=="@" else row['Loser Abbr'], axis=1)
df_games['Home Score'] = df_games.apply(lambda row: row['PtsL'] if row['LoserIsHome'] == "@" else row['PtsW'], axis=1)
df_games['Away Score'] = df_games.apply(lambda row: row['PtsW'] if row['LoserIsHome'] == "@" else row['PtsL'], axis=1)
df_games['Winner Yds'] = df_games.apply(lambda row: row['Winner Abbr'] if row['YdsW']>=row['YdsL'] else row['Loser Abbr'], axis=1)
df_games['Loser Yds'] = df_games.apply(lambda row: row['Winner Abbr'] if row['YdsW']<row['YdsL'] else row['Loser Abbr'], axis=1)
df_games['Margin Yds'] = df_games.apply(lambda row: abs(row['YdsW']-row['YdsL']), axis=1)
df_games['Winner TO'] = df_games.apply(lambda row: row['Winner Abbr'] if row['TOW']>=row['TOL'] else row['Loser Abbr'], axis=1)
df_games['Loser TO'] = df_games.apply(lambda row: row['Winner Abbr'] if row['TOW']<row['TOL'] else row['Loser Abbr'], axis=1)
df_games['Margin TO'] = df_games.apply(lambda row: abs(row['TOW']-row['TOL']), axis=1)
df_games['Margin Pts'] = df_games.apply(lambda row: abs(row['PtsW']-row['PtsL']), axis=1)
df_games['Total Pts'] = df_games.apply(lambda row: abs(row['PtsW']+row['PtsL']), axis=1)
df_games['Margin Pct'] = df_games.apply(lambda row: row['Margin Pts']-row['Total Pts'], axis=1)
df_games['home_team_winner'] = df_games.apply(lambda row: False if row['LoserIsHome']=="@" else True, axis=1)
df_games['away_win_bonus'] = df_games.apply(lambda row: 0 if row['home_team_winner'] else 1, axis=1)

print(df_games.head())

#print(df_games[df_games['Team Abbr_x'].isna()])
### Need to add error handling for unmapped abbreviations

   Week  Day        Date    Time                Winner LoserIsHome  \
0     1  Thu  2025-09-04  8:20PM   Philadelphia Eagles         NaN   
1     1  Fri  2025-09-05  8:00PM  Los Angeles Chargers         NaN   
2     1  Sun  2025-09-07  1:00PM  Jacksonville Jaguars         NaN   
3     1  Sun  2025-09-07  1:00PM   Pittsburgh Steelers           @   
4     1  Sun  2025-09-07  1:00PM    Cincinnati Bengals           @   

                Loser  PtsW  PtsL   YdsW  ...  Loser Yds  Margin Yds  \
0      Dallas Cowboys  24.0  20.0  302.0  ...        PHI         5.0   
1  Kansas City Chiefs  27.0  21.0  394.0  ...         KC        47.0   
2   Carolina Panthers  26.0  10.0  378.0  ...        CAR       123.0   
3       New York Jets  34.0  32.0  271.0  ...        PIT       123.0   
4    Cleveland Browns  17.0  16.0  141.0  ...        CIN       186.0   

   Winner TO  Loser TO  Margin TO Margin Pts Total Pts Margin Pct  \
0        DAL       PHI        1.0        4.0      44.0      -40.0   
1       

In [4]:
# Convert 'Date' column to datetime
df_games['DateTime'] = pd.to_datetime(df_games['Date'])

# Create a long-form DataFrame with team names and dates
home_teams = df_games[['DateTime', 'Home Team', 'season', 'week']].rename(columns={'Home Team': 'Team'})
away_teams = df_games[['DateTime', 'Away Team', 'season', 'week']].rename(columns={'Away Team': 'Team'})
all_games = pd.concat([home_teams, away_teams])

# Sort by team, season, and date
all_games = all_games.sort_values(by=['Team', 'season', 'DateTime'])

# Calculate days since last game for each team within each season
all_games['DaysSinceLastGame'] = all_games.groupby(['Team', 'season'])['DateTime'].diff().dt.days

# Assign a default value of days for Week 1 (to represent off-season) or for NaN (2017 season hurricane impacts)
all_games.loc[all_games['week'] == 1, 'DaysSinceLastGame'] = 21
all_games['DaysSinceLastGame'] = all_games['DaysSinceLastGame'].fillna(21)
all_games['DaysSinceLastGame'] = all_games['DaysSinceLastGame'].astype(int)

# Merge back with the original DataFrame to add the days since last game for both teams
df_games = df_games.merge(all_games.rename(columns={'Team': 'Home Team', 'DaysSinceLastGame': 'DaysSinceLastGame_Home'}),
              on=['DateTime', 'Home Team', 'season', 'week'], how='left')
df_games = df_games.merge(all_games.rename(columns={'Team': 'Away Team', 'DaysSinceLastGame': 'DaysSinceLastGame_Away'}),
              on=['DateTime', 'Away Team', 'season', 'week'], how='left')

#print(df_games.head())
print(df_games.columns)

Index(['Week', 'Day', 'Date', 'Time', 'Winner', 'LoserIsHome', 'Loser', 'PtsW',
       'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL', 'season', 'week', 'Winner Abbr',
       'Loser Abbr', 'Home Team', 'Away Team', 'Home Score', 'Away Score',
       'Winner Yds', 'Loser Yds', 'Margin Yds', 'Winner TO', 'Loser TO',
       'Margin TO', 'Margin Pts', 'Total Pts', 'Margin Pct',
       'home_team_winner', 'away_win_bonus', 'DateTime',
       'DaysSinceLastGame_Home', 'DaysSinceLastGame_Away'],
      dtype='object')


In [5]:
df_games.to_csv('../data/nfl_games_pfr.csv')